In [ ]:
import sympy as sym
m,g,delta,k,b,d,phi,c,R, l0,l1,alpha =sym.symbols('m, g, \delta, k, b, d, \phi, c, R, L_0, L_1, \alpha',positive=True)
x1e, x2e, x3e, ve = sym.symbols('x_1^e, x_^2e, x_3^e, v^e', real = True)
PHIT = 5/(7 * m) * (m*g*sym.sin(phi) + c*(x3e**2)/((delta - x1e)**2) - k*(x1e -d) - b*x2e) 
diffphi = sym.diff(PHIT,x1e)
print(sym.latex(diffphi),'\n')
print(diffphi,'\n')
diffphi2 = sym.diff(PHIT,x2e)
print(sym.latex(diffphi2),'\n')
print(diffphi2,'\n')
diffphi3 = sym.diff(PHIT,x3e)
print(sym.latex(diffphi3),'\n')
print(diffphi3,'\n')
y = delta - x1e
L = l0 + l1 * sym.exp(-alpha*y)
psi = (ve - x3e*R) / L
diffpsi = sym.diff(psi,x1e)
print(sym.latex(diffpsi),'\n')
print(diffpsi,'\n')
diffpsi2 = sym.diff(psi,x3e)
print(sym.latex(diffpsi2),'\n')
print(diffpsi2,'\n')
diffpsi3 = sym.diff(phi,ve)
print(sym.latex(diffpsi3),'\n')
print(diffpsi3,'\n')

- \frac{5 \left(- \frac{2 c \left(x^{e}_{3}\right)^{2}}{\left(\delta - x^{e}_{1}\right)^{3}} - k\right)}{7 m} 

-5*(-2*c*x_3^e**2/(\delta - x_1^e)**3 - k)/(7*m) 

\frac{5 b}{7 m} 

5*b/(7*m) 

\frac{10 c x^{e}_{3}}{7 m \left(\delta - x^{e}_{1}\right)^{2}} 

10*c*x_3^e/(7*m*(\delta - x_1^e)**2) 

- \frac{lpha L_{1} \left(- R x^{e}_{3} + v^{e}\right) e^{- lpha \left(\delta - x^{e}_{1}\right)}}{\left(L_{0} + L_{1} e^{- lpha \left(\delta - x^{e}_{1}\right)}\right)^{2}} 

-lpha*L_1*(-R*x_3^e + v^e)*exp(-lpha*(\delta - x_1^e))/(L_0 + L_1*exp(-lpha*(\delta - x_1^e)))**2 

- \frac{R}{L_{0} + L_{1} e^{- lpha \left(\delta - x^{e}_{1}\right)}} 

-R/(L_0 + L_1*exp(-lpha*(\delta - x_1^e))) 

0 

0 



In [ ]:
import numpy as np
from control import TransferFunction, margin
from multiprocessing import Pool, cpu_count


def compute_pd_values(args):
    """
    Computes the most optimized PID values and all possible PID values within a subrange of the ranges for Kp, Ti, and
    Td
    """
    Kp_range, Ti_range, Td_range, G = args
    all_pids = []
    best_pid = None
    best_margin = float('-inf')
    for Kp in Kp_range:
        for Ti in Ti_range:
            for Td in Td_range:
                if Kp == 0 and Ti == 0 and Td == 0:
                    continue
                pid = TransferFunction([Kp * Td * Ti, Kp * Ti, Kp], [Td, 1, 0])
                try:
                    gm, pm, sm, _ = margin(G * pid)
                except:
                    continue
                if sm > 0 and pm > 0:
                    all_pids.append((Kp, Ti, Td, sm))
                    if sm > best_margin:
                        best_pid = (Kp, Ti, Td, sm)
                        best_margin = sm
    return (best_pid, all_pids)


def find_best_pid(G, kp_range, ti_range, td_range, n_workers=(cpu_count()-2), ram_cap_gb=8):
    """
    Finds the most optimized PID values based on stability margins, using multicore processing and RAM cap
    """
    total_range = len(kp_range) * len(ti_range) * len(td_range)
    max_size = int(ram_cap_gb * 1024 * 1024 * 1024 / 8)  # 8 bytes per float
    chunk_size = int(max_size / (n_workers * 4))
    chunks = [(kp_range, ti_range, td_range[i:i + chunk_size], G) for i in range(0, len(td_range), chunk_size)]
    with Pool(n_workers) as pool:
        results = pool.map(compute_pd_values, chunks)
    all_pids = []
    for result in results:
        all_pids.extend(result[1])
    all_pids.sort(key=lambda x: -x[3])  # Sort by margin, descending
    best_pid = None
    best_margin = float('-inf')
    for result in results:
        if result[0] is not None:
            pid = result[0]
            margin = pid[3]
            if margin > best_margin:
                best_pid = pid
                best_margin = margin
    return (best_pid, all_pids)


# Define system parameters
m = 0.462
g = 9.81
d = 0.42
dirac = 0.65
r = 0.123
R = 2200
lo = 123 * 10 ** -3
li = 24.1 * 10 ** -3
alpha = 1.2
c = 6.811
k = 1885
b = 10.4
phi = 41
tm = 0.03

#eqquilibrium values:
x1e = 0.5
x2e = 0
x3e = np.sqrt(((-m*g*np.sin(phi) - b*x2e - k(x1e-d))*(dirac - x1e)**2)/c)
ve = x3e*R

#Reused equations
expo = np.exp(-alpha * (dirac - x1e))
L = lo + li * expo
s = ctrl.TransferFunction.s
A = (5/(7*m)) * (-k-(2*c*(x3e** 2)) / ((dirac -(x1e))**3))
B = -(5 * b) / (7 * m)
C = (10 * c * x3e) / (7 * m * (dirac - x1e) ** 2)
D = -(li * alpha * (-R * x3e + ve) * expo) / L ** 2
E = R / L
F = 1 / L
G = (F*C)/((s**2 - A -B*s)*(s-E))

if __name__ == '__main__':
    kp_range = np.arange(0, 1000, 1)
    ti_range = np.arange(0, 1000, 1)
    td_range = np.arange(0, 1000, 1)
    best_pid, all_pids = find_best_pid(G, kp_range, ti_range, td_range, n_workers=4, ram_cap_gb=8)
    print(f"Best PID: {best_pid}")
    print(f"All PID values sorted by margin: {all_pids}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import control as ctrl

# Define system parameters
m = 0.462
g = 9.81
d = 0.42
dirac = 0.65
r = 0.123
R = 2200
lo = 123 * (10 ** -3)
li = 24.1 * (10 ** -3)
alpha = 1.2
c = 6.811
k = 1885
b = 10.4
phi = 41
tm = 0.03

#eqquilibrium values:
x1e = 0.5
x2e = 0
x3e = sp.N(sp.N(sp.sqrt(((sp.N(-m*g*sp.sin(phi))- (b*x2e) - (k*(x1e-d)))*(dirac-x1e)**2)/c)))
ve = x3e*R
print(x3e)
print(sp.N(-m*g*sp.sin(phi)))
print(sp.N(-m*g*sp.sin(phi))- (b*x2e))
print((k*(x1e-d)))
print(sp.N(-m*g*sp.sin(phi))- (b*x2e) - (k*(x1e-d)))
#Reused equations
expo = np.exp(-alpha * (dirac - x1e))
L = lo + li * expo
s = ctrl.TransferFunction.s
A = (5/(7*m)) * (-k-(2*c*(x3e** 2)) / ((dirac -(x1e))**3))
B = -(5 * b) / (7 * m)
C = (10 * c * x3e) / (7 * m * (dirac - x1e) ** 2)
D = -(li * alpha * (-R * x3e + ve) * expo) / L ** 2
E = R / L
F = 1 / L
G = (F*C)/((s**2 - A -B*s)*(s-E))

print(A)
print(B)
print(C)
print(D)
print(E)
print(F)



0.704123440596659*I
0.718912832010078
0.718912832010078
150.80000000000004
-150.081087167990


TypeError: ignored

In [ ]:
!pip install control

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.8/432.8 kB 6.1 MB/s eta 0:00:00
